# Creando un dataset


Esta es probablemente la parte más interesante del tutorial, donde podrás aprovechar las herramientas EOTDL para crear un nuevo conjunto de datos. Aquí cubrimos:

1. **Exploración de datos**: Dada una área de interés, consulta los datos centinela disponibles para tu conjunto de datos.
2. **Acceso a los datos**: descarga los datos para crear el conjunto.
3. **Preparación de los datos**: limpia los datos, realiza la ingeniería de características, el análisis de datos, el etiquetado, etc.

Una vez que tu conjunto de datos esté listo, puedes incorporarlo al EOTDL como vimos en el cuaderno anterior y empezar a trabajar con él como cualquier otro conjunto de datos del repositorio.


## Exploración


En primer lugar, exploremos el área de interés que hemos seleccionado para este taller. En este caso, hemos elegido el [Embalse de Boadella](https://es.wikipedia.org/wiki/Embalse_de_Darnius_Boadella) en Cataluña, España, cuya geometría se encuentra en la carpeta de datos `workshop_data/boadella.geojson`. Aquí utilizamos [leafmap](https://leafmap.org/) para visualizarlo, pero puedes usar la solución que prefieras.


In [ ]:
# !uv add leafmap

In [ ]:
import leafmap
import geopandas as gpd

in_geojson = "workshop_data/boadella.geojson"
gdf = gpd.read_file(in_geojson)

centroid_coords = gdf["geometry"].centroid
centroid = [
    centroid_coords.y.values[0],
    centroid_coords.x.values[0],
]  # We are going to use the centroid later

m = leafmap.Map(center=centroid, zoom=13)
m.add_geojson(in_geojson, layer_name="Boadella reservoir")
m

Al crear conjuntos de datos compatibles con IA, es habitual trabajar con una resolución fija. Se pueden recuperar escenas completas y cortar fragmentos, o usar la función EOTDL para generar los cuadros delimitadores adecuados. Para que todas las imágenes del conjunto de datos tengan 512x512 píxeles, usaremos el centroide extraído previamente del geoJSON y generaremos un cuadro delimitador que generará una imagen de 512x512 píxeles con una resolución de 10 m, ya que usaremos datos de Sentinel.


In [ ]:
from eotdl.tools import bbox_from_centroid

boadella_bbox = bbox_from_centroid(
    x=centroid[0], y=centroid[1], pixel_size=10, width=512, height=512
)
boadella_bbox

¡Visualicemos el cuadro delimitador en un mapa!


In [ ]:
from eotdl.tools import bbox_to_polygon

# Create a polygon from the bbox
boadella_polygon = bbox_to_polygon(boadella_bbox)
# Create a GeoDataFrame from the polygon
gdf = gpd.GeoDataFrame(geometry=[boadella_polygon])
# Save the bounding box as a geoJSON file, if needed
gdf.to_file(
    "workshop_data/boadella_bbox.geojson", driver="GeoJSON"
)  # Uncomment to save the bbox as a GeoJSON file

m.add_geojson("workshop_data/boadella_bbox.geojson", layer_name="Boadella bbox")
m

Ahora que tenemos el cuadro delimitador deseado, podemos buscar imágenes de Sentinel-2 disponibles en él. Esto se puede hacer a través del EOTDL.

Primero, podemos buscar qué sensores Sentinel son compatibles con el EOTDL.


In [ ]:
from eotdl.access import SUPPORTED_COLLECTION_IDS

SUPPORTED_COLLECTION_IDS

Si queremos buscar imágenes de Sentinel-2 disponibles en nuestra área de interés, debemos definir un rango de fechas para buscarlas. Ya hemos definido un intervalo de tiempo para este taller, que se encuentra en el archivo `workshop_data/dates.csv`.


In [ ]:
import csv

dates = list()
with open("workshop_data/dates.csv", "r") as file:
    reader = csv.reader(file)
    for row in reader:
        dates.append(row[0])
dates.sort()

dates[:5]

Aunque tenemos las fechas específicas, vamos a buscar todo el intervalo de tiempo, tal como un demostrador.


> Usamos Sentinel Hub internamente, por lo que necesitarás las credenciales adecuadas. Puedes generarlas automáticamente desde tu perfil de usuario (https://www.eotdl.com/profile) aceptando los términos y condiciones. Al iniciar sesión en EOTDL, a través de la biblioteca o la CLI, recuperamos y almacenamos esta información, así que no tienes que preocuparte. Sin embargo, hay un par de inconvenientes: <br><br> 1. Si ya tienes una cuenta de Sentinel HUB con el mismo correo electrónico que tu cuenta de EOTDL, deberás recuperar las credenciales del Panel de Control de Sentinel Hub y configurarlas como variables de entorno. <br> 2. Las credenciales generadas mediante EOTDL pueden caducar después de un tiempo (estamos trabajando en ello). Si esto ocurre, avísanos en Discord para solucionar el problema. <br><br> En cualquier caso, puedes proporcionar tus propias credenciales configurando las variables de entorno adecuadas: `SH_CLIENT_ID` y `SH_CLIENT_SECRET`.


In [ ]:
from eotdl.access import search_sentinel_imagery

time_interval = (dates[0], dates[-1])

r = search_sentinel_imagery(time_interval, boadella_bbox, "sentinel-2-l2a")
response = list(r)
response[:5]

Tienen sentido, ya que el [tiempo de revisita](https://docs.sentinel-hub.com/api/latest/data/sentinel-2-l2a/#basic-facts) para Sentinel-2 es de 5 días.

Por último, verifiquemos la cantidad de fechas con imágenes disponibles.


In [ ]:
print(len(response))

To sum up this section, we have explored our AoI, generated a bounding box and a time interval in which to look for imagery and searched for Sentinel-2 imagery.


## Descargar


El siguiente paso es descargar las imágenes. Podemos descargarlas una por una, como se indica a continuación.


In [ ]:
from eotdl.access import download_sentinel_imagery

first_date = dates[0]

# Uncomment to demonstrate
download_sentinel_imagery(
    "data/sentinel_2", first_date, boadella_bbox, "sentinel-2-l2a"
)

Por otro lado, podemos buscar y descargar todas las imágenes disponibles dentro de un intervalo de tiempo, como se indica a continuación. Este es el método recomendado para descargas masivas, pero tiene la desventaja de que no podemos controlar la calidad de las imágenes, por ejemplo, conocer su cobertura de nubes.


In [ ]:
# Uncomment to demonstrate

demostration_dates = (dates[0], dates[2])

download_sentinel_imagery(
    output="data/sentinel_2",
    time_interval=demostration_dates,
    bounding_box=boadella_bbox,
    collection_id="sentinel-2-l2a",
)

¡Eso es todo! Hemos descargado las imágenes de nuestro conjunto de datos. ¡Vamos a revisarlas!


In [ ]:
from glob import glob

rasters = glob("data/sentinel_2/*.tiff")
rasters[:5]

Un último paso opcional es cambiar el nombre de las imágenes y limpiar el directorio.


In [ ]:
files = glob('data/sentinel_2/*')
for file in files:
    new_file_name = file.replace('sentinel-2-l2a', 'Boadella').replace('.tiff', '.tif')
    ! mv $file $new_file_name

!rm -r data/sentinel_2/*.json

In [ ]:
!ls data/sentinel_2

## Oportunidades de discusión y contribución


No dudes en hacer preguntas ahora (en directo o a través de Discord) y sugerir mejoras futuras.

- ¿Qué funciones de exploración de datos te gustaría ver?
- ¿Qué otras funciones de descarga de datos te gustaría ver?
- ¿Qué funciones y herramientas de preparación de datos te gustaría ver?
- ¿Cómo es tu flujo de trabajo habitual?
